In [2]:
# 딥러닝을 이용한 자연어 처리
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
import numpy as np

In [4]:
text = '해보지 않으면 해낼 수 없다'
# text_to_word_sequence => 빈 공간 기준 split
result = text_to_word_sequence(text)
print(result)

['해보지', '않으면', '해낼', '수', '없다']


In [5]:
docs = [
    '먼저 텍스트의 각 단어를 나누어 토큰화 합니다.',
    '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
    '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.',
]

In [6]:
token = Tokenizer()
token.fit_on_texts(docs)

In [7]:
# 각 단어의 빈도수 측정
print(token.word_counts)

OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 1), ('합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])


In [8]:
# 문장이 몇개있는지 측정
print(token.document_count)

3


In [9]:
# 각 단어가 몇개의 문장에 등장하는가
print(token.word_docs)

defaultdict(<class 'int'>, {'합니다': 1, '각': 1, '텍스트의': 2, '토큰화': 1, '먼저': 1, '단어를': 1, '나누어': 1, '토큰화해야': 1, '단어로': 1, '인식됩니다': 1, '딥러닝에서': 2, '결과는': 1, '토큰화한': 1, '있습니다': 1, '사용할': 1, '수': 1})


In [10]:
# parsing한 문장을 단어별로 tokenize 하면서 indexing, 단 인덱싱은 1번부터 시작함
print(token.word_index)

{'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화': 7, '합니다': 8, '단어로': 9, '토큰화해야': 10, '인식됩니다': 11, '토큰화한': 12, '결과는': 13, '사용할': 14, '수': 15, '있습니다': 16}


In [14]:
# one hot encoding으로 conversion 
text = '오랫동안 꿈꾸는 이는 그 꿈을 닮아간다'
token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [16]:
# 딥러닝데이터는 수치로 입력해야함
# 그래서 문장을 수치로 바꿀 필요성이 있는데 어떻게 바꾸느냐? => 텍스트를 시퀸지스로 바꾸어줌
x = token.texts_to_sequences([text])
print(x)

[[1, 2, 3, 4, 5, 6]]


In [17]:
# 이번에는 단어 하나하나를 one hot encoding
# 단어를 원핫인코딩 할때는 차원을 하나 늘려야함
word_size = len(token.word_index) + 1 
# to_categorical(x, 항목이 몇개인지 판단)
# 7차원을 만들기 위해 x + 1 인 word_size로 7차원 생성
x = to_categorical(x, num_classes=word_size)
print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


In [21]:
# 긍정 부정 예측
docs =[
    "너무 재밌네요","최고예요","참 잘 만든 영화예요",
    "추천하고 싶은 영화입니다","한번 더 보고싶네요",
    "글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"
]
classes = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])
token = Tokenizer()
token.fit_on_texts(docs)
# 모든 단어에 인덱싱 붙임
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [23]:
# 이 문장을 word_index와 token.texts_to_sequences를 사용해 숫자로 변경
x = token.texts_to_sequences(docs)
print(x)

[[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [24]:
# 기본적으로 딥러닝은 input 차원이 고정되어있어야함
# 단어는 input이 다양하므로 어떻게 맞춰주느냐 -> pad_sequences을 이용해서 padding
# 위의 단어중에 제일 긴놈이 4이므로 4로 세팅
# 길이가 4보다 낮은놈은 어떻게 할거냐? default 는 0으로 땜빵
padded_x = pad_sequences(x, 4)
print(padded_x)

[[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [27]:
# 모델 생성
model = models.Sequential()
# 기본적으로 n차원은 중심부로부터 n차원 특징들의 제곱합의 루트
# ex) 300차원 >> 10차원으로 변형 할 때 word2vec [고차원의 벡터를 저차원의 밀집 벡터로 변환(.)]
# keras에서 이와같은 역할을 하는게 layers.Embedding
# embedding layer 사용


# 실제로 단어의 종류는 20개(input), output=8
# 20 * 8 + 8 = 168 
model.add(layers.Embedding(len(token.word_index) + 1, 8, input_length=4))
# Flatten으로 누름 4 * 8 = 32
model.add(layers.Flatten())
# 긍정이냐 부정이냐로 판단하기위해 dense 1
# 32 * 1 + 1 = 33
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 8)              168       
                                                                 
 flatten_1 (Flatten)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)

Epoch 1/20
1/1 [==============================] - 1s 624ms/step - loss: 0.7007 - accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6985 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6964 - accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6943 - accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6921 - accuracy: 0.5000
Epoch 6/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6900 - accuracy: 0.4000
Epoch 7/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6879 - accuracy: 0.5000
Epoch 8/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6858 - accuracy: 0.6000
Epoch 9/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6836 - accuracy: 0.7000
Epoch 10/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6815 - accuracy: 0.7000
Epoch 11/20
1/1 [====================